### Code to Compare Model to a Benchmark Created Using the Bottom N Percentile of Points

Note: code uses datasets from manning_baselines.ipynb so ensure that is ran first

In [ ]:
import sys
sys.path.append('../')
import functions as f

model_name = 'rf_model_MHD.joblib'
site, site_name, compound, model = f.access_info(model_name)

print(f"Evaluating a \033[1m{model.__class__.__name__}\033[0;0m model on \033[1m{compound}\033[0;0m at \033[1m{site_name}\033[0;0m.")

#### Results

In [ ]:
percentile = 10

df_benchmark = f.calc_benchmark(percentile)
f.plot_benchmark(df_benchmark, percentile)

In [ ]:
f.compare_benchmark_to_model(df_benchmark, percentile, model, 2015, 2020)